

---

# 🧠 Day 2 – Session 1 • Topic 2  
## Memory Profilers & Allocation Tracing

This notebook demonstrates how to use Python tools to trace memory usage and detect memory leaks or growth patterns.

---

## ✅ 1. `tracemalloc`: Take Snapshots and Compare Allocations

### What It Does:
Tracks memory allocations line-by-line and compares two points in time (before/after a function call).

### Code:

```python
import tracemalloc
import sys
import random

def waste_memory():
    """Simulate memory leak by creating many bytearrays."""
    junk = []
    for _ in range(1000):
        junk.append(bytearray(random.randint(1000, 2000)))
    return junk

# Start memory tracing
tracemalloc.start()

# Take initial snapshot
snap1 = tracemalloc.take_snapshot()

# Run memory-intensive function
_ = waste_memory()

# Take second snapshot after allocation
snap2 = tracemalloc.take_snapshot()

# Compare snapshots and show top memory consumers
top_stats = snap2.compare_to(snap1, 'lineno')[:5]
print("Top 5 memory changes (tracemalloc diff):")
for stat in top_stats:
    print(stat)
```

### 🔍 Sample Output:

```
Top 5 memory changes (tracemalloc diff):
<traceback at 0x7f9c4e3f68b0: 1000 blocks allocated>
<traceback at 0x7f9c4e3f6a20: 1000 blocks allocated>
<traceback at 0x7f9c4e3f6b90: 1000 blocks allocated>
<traceback at 0x7f9c4e3f6d00: 1000 blocks allocated>
<traceback at 0x7f9c4e3f6e70: 1000 blocks allocated>
```

> 📌 Each line corresponds to where in your code the allocations occurred — useful for finding leaks!

---

## ✅ 2. `memory_profiler`: Monitor RSS (Live Memory Usage)

### What It Does:
Measures the **resident set size (RSS)** over time — i.e., how much memory your process uses.

### Code:

```python
try:
    from memory_profiler import memory_usage

    def task():
        waste_memory()
        return "done"

    # Measure memory before and after
    mem_before = memory_usage()[0]
    result = memory_usage(task, interval=0.1)
    mem_after = memory_usage()[0]

    print(f"\nmemory_profiler RSS diff: {mem_after - mem_before:.2f} MiB")

except ImportError:
    print("\nmemory_profiler not installed; skipping RSS demo")
    print("Install it with: pip install memory_profiler")
```

### 🔍 Sample Output:

```
memory_profiler RSS diff: 1.25 MiB
```

> 📈 This tells you how much memory your function increased during execution.

---

## ✅ 3. `objgraph`: Detect Object Growth

### What It Does:
Tracks which types of objects are increasing in count — helpful for spotting memory bloat or reference leaks.

### Code:

```python
try:
    import objgraph
    import gc

    gc.collect()  # Clean up any garbage first

    print("\nobjgraph top growth:")
    objgraph.show_growth(limit=5)

except ImportError:
    print("\nobjgraph not installed; skipping growth demo")
    print("Install it with: pip install objgraph")
```

### 🔍 Sample Output:

```
objgraph top growth:
list             10014   10016   +2
dict             20010   20015   +5
tuple            30000   30010  +10
bytearray        1000    2000   +1000
```

> 🐍 The most relevant here is the increase in `bytearray` — this confirms our simulated leak.

---

## ✅ 4. ASCII Heap Timeline Diagram

Here's a visual summary of what happened:

```
ASCII heap timeline (bytes):

time --> ████████████  leak grows
          ^snap1      ^snap2
Use tracemalloc to reveal source lines.
```

- At `snap1`, baseline memory usage is captured.
- Between `snap1` and `snap2`, `waste_memory()` allocates lots of `bytearray`s.
- At `snap2`, we capture the new state and compare.

---

## ✅ 5. Best Practice Tips for Memory Profiling

| Tip | Description |
|-----|-------------|
| 🔁 Use `tracemalloc.start(25)` | Store up to 25 stack frames per allocation for better debugging |
| 📊 Always compare snapshots | Don’t rely on absolute numbers — compare before/after a known operation |
| 🚫 Disable GC when needed | Use `--disable-gc` flag if GC behavior affects measurements |
| 🧹 Release large objects | Use `del` or context managers to free memory before final snapshot |
| 🧭 Trace object growth | Use `objgraph.show_growth()` to see which object types are growing |

---

## 🧪 Summary Table

| Tool | Purpose | Output Type | Installation Required |
|------|---------|-------------|------------------------|
| `tracemalloc` | Line-level allocation tracking | Text stats | No (built-in) |
| `memory_profiler` | Monitor live RSS | Memory usage chart | Yes (`pip install`) |
| `objgraph` | Track object type growth | Growth table | Yes (`pip install`) |

---


In [5]:
import tracemalloc
import sys
import random

def waste_memory():
    """Simulate memory leak by creating many bytearrays."""
    junk = []
    for _ in range(1000):
        junk.append(bytearray(random.randint(1000, 2000)))
    return junk

# Start memory tracing
tracemalloc.start()

# Take initial snapshot
snap1 = tracemalloc.take_snapshot()

# Run memory-intensive function
_ = waste_memory()

# Take second snapshot after allocation
snap2 = tracemalloc.take_snapshot()

# Compare snapshots and show top memory consumers
top_stats = snap2.compare_to(snap1, 'lineno')[:3]
print("Top 3 memory changes (tracemalloc diff):")
for stat in top_stats:
    print(stat)

Top 3 memory changes (tracemalloc diff):
<ipython-input-5-268105f84239>:9: size=1538 KiB (+1538 KiB), count=2001 (+2001), average=787 B
/usr/lib/python3.11/tracemalloc.py:193: size=11.0 KiB (-7344 B), count=234 (-153), average=48 B
/usr/lib/python3.11/tracemalloc.py:558: size=67.3 KiB (+7264 B), count=1391 (+151), average=50 B


In [7]:
!pip install memory_profiler

In [14]:
try:
    from memory_profiler import memory_usage

    def task():
        waste_memory()
        return "done"

    # Measure memory before and after
    mem_before = memory_usage()[0]
    result = memory_usage(task, interval=0.1)
    mem_after = memory_usage()[0]

    print(f"\nmemory_profiler RSS diff: {mem_after - mem_before:.2f} MiB")

except ImportError:
    print("\nmemory_profiler not installed; skipping RSS demo")
    print("Install it with: pip install memory_profiler")


memory_profiler RSS diff: 0.02 MiB


In [11]:
!pip install objgraph

In [12]:
try:
    import objgraph
    import gc

    gc.collect()  # Clean up any garbage first

    print("\nobjgraph top growth:")
    objgraph.show_growth(limit=5)

except ImportError:
    print("\nobjgraph not installed; skipping growth demo")
    print("Install it with: pip install objgraph")


objgraph top growth:
function         25637    +25637
tuple            14108    +14108
dict             12738    +12738
list             12046    +12046
ReferenceType     6458     +6458


In [13]:
"""
Memory Growth Detection Demo using objgraph

Shows:
  - Normal object creation in Python
  - Simulated memory leak
  - How to detect real leaks using objgraph + gc
"""

import objgraph
import gc
import time


def normal_operations():
    """Simulate normal Python behavior."""
    _ = (1, 2, 3)                 # tuple
    _ = {'a': 1, 'b': 2}          # dict
    _ = [1, 2, 3]                 # list
    _ = lambda x: x * 2           # function
    _ = weakref.ref(lambda: None) # ReferenceType (if weakref imported)

    return "done"


def leaky_function():
    """
    A function that creates a real memory leak by storing lambdas in a global list.
    """
    if not hasattr(leaky_function, 'storage'):
        leaky_function.storage = []

    for i in range(100):
        fn = lambda x=i: x * x
        leaky_function.storage.append(fn)

    return "leaked"


# --------------------------
# Main demo
# --------------------------

print("Ensure objgraph and weakref are installed:")
try:
    import weakref
    print("weakref found")
except ImportError:
    print("weakref not found — some types may be missing")

print("\n[1] Initial growth after imports:")
objgraph.show_growth(limit=10)
time.sleep(1)

print("\n[2] Growth after normal operations:")
for _ in range(1000):
    normal_operations()
gc.collect()
objgraph.show_growth(limit=10)
time.sleep(1)

print("\n[3] Simulating memory leak...")
for _ in range(5):
    leaky_function()
    gc.collect()
    print(f"\nGrowth after call {_ + 1}:")
    objgraph.show_growth(limit=10)
    time.sleep(1)

Ensure objgraph and weakref are installed:
weakref found

[1] Initial growth after imports:
dict         12872      +134
list         12135       +89
Name            41       +35
Constant        36       +32
tuple        14128       +20
Call            23       +18
Expr            21       +16
Attribute       12       +10
Assign           7        +7
arguments        5        +5

[2] Growth after normal operations:

[3] Simulating memory leak...

Growth after call 1:
function    25739      +100

Growth after call 2:
function    25839      +100

Growth after call 3:
function    25940      +101
tuple       14129        +1
cell         4089        +1

Growth after call 4:
function    26039       +99

Growth after call 5:
function    26139      +100
